# LEVEL 10 - EDA

In [42]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA


from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [43]:
# Load CSV file into a pandas DataFrame
file_path = "./data/train_data.csv"
data = pd.read_csv(file_path)
# Removing wrong values
data.dropna(inplace=True)
data = data[data['AMPS'].between(0, 1)]
data['UNIT'][data['TEMP']>100] = 'K'
data['UNIT'][data['TEMP']<100] = 'C'
data['TEMP'][data['TEMP']>100] = data['TEMP'][data['TEMP']>100] - 273.25
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   MODE    971 non-null    object 
 1   POWER   971 non-null    object 
 2   AMPS    971 non-null    float64
 3   VOLTS   971 non-null    float64
 4   TEMP    971 non-null    float64
 5   UNIT    971 non-null    object 
 6   DELTA   971 non-null    float64
 7   GAMMA   971 non-null    float64
 8   OUTPUT  971 non-null    float64
dtypes: float64(6), object(3)
memory usage: 68.4+ KB


## Encoding categorical variables

In [44]:
encoder = OneHotEncoder(sparse=False)

encoded_data = encoder.fit_transform(data[['POWER', 'MODE', 'UNIT']])
df_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names(['POWER', 'MODE', 'UNIT']))

data = pd.concat([data, df_encoded], axis=1)
data = data.drop(['MODE', 'POWER', 'UNIT'], axis=1)
data

,AMPS,VOLTS,TEMP,DELTA,GAMMA,OUTPUT,POWER_high,POWER_low,MODE_REDACTED,MODE_auto,MODE_beam,MODE_burst,UNIT_C,UNIT_K
0,0.585,4.347,66.812,-0.659,-0.527,3.749,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.687,2.412,-17.044,0.508,-0.371,-1.062,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.888,2.979,40.918,0.600,1.412,2.265,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.114,0.754,-43.308,-0.760,-0.109,1.847,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.318,3.955,37.752,-0.751,0.492,1.821,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,0.350,4.257,1.331,0.991,-0.335,2.466,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
967,0.644,1.128,-27.707,-0.529,0.034,-1.834,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
968,0.887,1.418,-71.690,0.729,-0.503,-0.188,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
969,0.323,0.291,-17.162,-0.120,0.524,0.081,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


## Testing models

In [45]:
X = data
Y = X['OUTPUT']
X = X.drop('OUTPUT', axis = 1)

kf = KFold(n_splits=12, random_state=42, shuffle=True)

In [46]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               1792      
                                                                 
 dense_11 (Dense)            (None, 256)               33024     
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dense_13 (Dense)            (None, 256)               65792     
                                                                 
 dense_14 (Dense)            (None, 1)                 257       
                                                                 
Total params: 166,657
Trainable params: 166,657
Non-trainable params: 0
_________________________________________________________________


In [47]:
NN_model.fit(X, Y, epochs=500, batch_size=32, validation_split = 0.2)

Epoch 1/500


25/25 [==============================] - 3s 20ms/step - loss: 6.0047 - mean_squared_error: 6.0047 - val_loss: 5.9836 - val_mean_squared_error: 5.9836
Epoch 2/500
25/25 [==============================] - 0s 10ms/step - loss: 5.8784 - mean_squared_error: 5.8784 - val_loss: 5.8562 - val_mean_squared_error: 5.8562
Epoch 3/500
25/25 [==============================] - 0s 10ms/step - loss: 5.7259 - mean_squared_error: 5.7259 - val_loss: 6.9842 - val_mean_squared_error: 6.9842
Epoch 4/500
25/25 [==============================] - 0s 9ms/step - loss: 5.8918 - mean_squared_error: 5.8918 - val_loss: 5.6289 - val_mean_squared_error: 5.6289
Epoch 5/500
25/25 [==============================] - 0s 10ms/step - loss: 5.6331 - mean_squared_error: 5.6331 - val_loss: 5.3966 - val_mean_squared_error: 5.3966
Epoch 6/500
25/25 [==============================] - 0s 10ms/step - loss: 5.3990 - mean_squared_error: 5.3990 - val_loss: 5.6040 - val_mean_squared_error: 5.6040
Epoch 7/500
25/25 [======================

In [48]:
file_path = "./data/test_data.csv"
df2 = pd.read_csv(file_path)
# Transform the test data using the same encoder instance
encoded_test_data = encoder.transform(df2[['POWER', 'MODE', 'UNIT']])

df_encoded_test  = pd.DataFrame(encoded_test_data, columns=encoder.get_feature_names(['POWER', 'UNIT', 'MODE']))
df2 = pd.concat([df2, df_encoded_test], axis=1)

df2 = df2.drop(['MODE', 'POWER', 'UNIT'], axis=1)
df2

,AMPS,VOLTS,TEMP,DELTA,GAMMA,POWER_high,POWER_low,UNIT_REDACTED,UNIT_auto,UNIT_beam,UNIT_burst,MODE_C,MODE_K
0,0.126,7.489,14.284,-0.818,0.692,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.917,4.557,17.746,0.251,-0.709,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.508,2.928,-9.866,-0.273,1.056,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.832,2.464,-32.047,0.145,-0.572,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.501,2.711,1.349,0.431,-0.939,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.639,2.318,-21.235,-0.786,0.399,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2496,0.630,1.569,-12.947,0.768,-0.001,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2497,0.813,1.854,-29.502,-0.408,0.638,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2498,0.172,3.663,-0.141,0.595,0.995,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [49]:
predictions = NN_model.predict(df2)

79/79 [==============================] - 0s 2ms/step


In [50]:
np.savetxt('./output/output_DNN.csv', predictions)